In [1]:
import json
import random

In [2]:
# chapters_list.json
json_path = './data/json/chapters_list.json'
with open(json_path, 'r', encoding='utf-8') as j :
    chapters = json.load(j)

In [3]:
chapters[0]

{'chapter_id': 1,
 'chapter_name': '몇일까요 (1)',
 'school_level': '초등',
 'grade_level': '초1',
 'semester': '1학기',
 'chapter_main': '',
 'chapter_sub': '9까지의 수'}

In [4]:
chapter_sub_list = [chapter['chapter_sub'] for chapter in chapters]
# 중복제거 - 이렇게 하면 셀 돌릴 때마다 내용물 순서 바뀔 수 있음
# chapter_sub_list = list(set(chapter_sub_list))
# len(chapter_sub_list)
# 순서 유지하면서 중복제거하기
from collections import OrderedDict
chapter_sub_list = list(OrderedDict.fromkeys(chapter_sub_list))

In [5]:
# 키:중단원, 밸류:그에 해당하는 데이터들
data_dic = {}
for chapter_sub in chapter_sub_list :
    name_list = []
    for chapter in chapters:
        if chapter_sub==chapter['chapter_sub']:
            name_list.append(chapter['chapter_name'])
            school_level = chapter['school_level']
            grade_level = chapter['grade_level']
            semester = chapter['semester']
    name_list = list(set(name_list))
    data_dic[chapter_sub]= [name_list, school_level, grade_level, semester]
data_dic['9까지의 수']

[['몇째일까요',
  '어느 수가 더 클까요',
  '몇일까요 (2)',
  '몇일까요 (1)',
  '1만큼 더 큰 수와 1만큼 더 작은 수는 무엇일까요',
  '수의 순서를 알아볼까요'],
 '초등',
 '초1',
 '1학기']

### ㅇ insert_diag_tests.sql

In [6]:
# insert_diag_tests.sql

file_path = "../api/sql/insert_diag_tests.sql"

with open(file_path, "w", encoding='utf-8') as file:
    for chapter_sub, value in data_dic.items() :
        for i in range(1, 6):
            sql = f"INSERT INTO tests (test_name, test_comments, test_school_level, test_grade_level, test_semester) VALUES ('{chapter_sub}({i})', \"{value[0]}을 알고 있는지 확인할 수 있는 진단학습지입니다.\", '{value[1]}', '{value[2]}', '{value[3]}');"
            file.write(sql + "\n")

print("SQL 문장들 insert_diag_tests.sql 파일로 저장 성공")

SQL 문장들 insert_diag_tests.sql 파일로 저장 성공


### ㅇ insert_diag_items.sql

In [7]:
# concepts_list.json
json_path = './data/json/concepts_list_v1.json'
with open(json_path, 'r', encoding='utf-8') as j :
    concepts = json.load(j)

In [8]:
concepts[0]

{'id': 5814,
 'name': '1부터 5까지의 수',
 'description': '1부터 5까지의 수를 읽고 셀 수 있다.',
 'school_level': '초등',
 'grade_level': '초1',
 'semester': '1학기',
 'chapter_id': 1,
 'chapter_main': '9까지의 수',
 'chapter_sub': '몇일까요 (1)',
 'chapter_subsub': '',
 'achievement_id': 1,
 'achievement_name': '50까지의수 개념을이해 하고, 수를 세고 읽고 쓸 수 있다.',
 'skill_id': 8}

In [9]:
# 이 데이터는 초등에 대해 main->sub으로 옮겨줘야 해
for concept in concepts:
    if concept['school_level']=='초등':
        concept['chapter_sub']=concept['chapter_main']

In [10]:
# items, tests_items에 필요한 데이터 추출
result_list = []
cnt=0
for i, chapter_sub in enumerate(chapter_sub_list):
    num=0
    i*=5
    for concept in concepts:
        if chapter_sub == concept['chapter_sub']:
            num+=1
            for j in range(1, 6):
                data = {}
                cnt+=1
                data['item_image_path'] = f"/images/items/diag/{i+j}({j})/{num}"
                data['concept_id'] = concept['id']
                data['test_id'] = i+j
                data['item_id'] = cnt
                data['test_item_number'] = num
                result_list.append(data)

In [11]:
result_list[0]

{'item_image_path': '/images/items/diag/1(1)/1',
 'concept_id': 5814,
 'test_id': 1,
 'item_id': 1,
 'test_item_number': 1}

In [12]:
result_list[8149]

{'item_image_path': '/images/items/diag/715(5)/13',
 'concept_id': 3352,
 'test_id': 715,
 'item_id': 8150,
 'test_item_number': 13}

In [13]:
file_path = "../api/sql/insert_diag_items.sql"

with open(file_path, "w") as file:
    for data in result_list:
        rand_num = random.randint(1, 5)
        item_image_path = data['item_image_path']
        concept_id = data['concept_id']
        sql = f"INSERT INTO items(item_answer, item_image_path, concept_id) VALUES ('{rand_num}', '{item_image_path}', {concept_id});"
        file.write(sql + "\n")

print("SQL 문장들 insert_diag_items.sql 파일로 저장 성공")

SQL 문장들 insert_diag_items.sql 파일로 저장 성공


### ㅇ insert_diag_testsitems.sql

In [14]:
file_path = "../api/sql/insert_diag_testsitems.sql"
        
with open(file_path, "w") as file:
    for data in result_list:
        test_id = data['test_id']
        item_id = data['item_id']
        test_item_number = data['test_item_number']
        sql = f"INSERT INTO tests_items(test_id, item_id, test_item_number) VALUES ({test_id}, {item_id}, {test_item_number});"
        file.write(sql + "\n")

print("SQL 문장들 insert_diag_testsitems.sql 파일로 저장 성공")

SQL 문장들 insert_diag_testsitems.sql 파일로 저장 성공
